In [ ]:
# import the package 
import cobra
#import cobra.test
import pandas as pd
import numpy as np
import requests
pd.options.display.max_rows = 4000
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [ ]:
ipfa = cobra.io.load_matlab_model("ipfa.mat")

In [ ]:
cobra.io.save_json_model(ipfa,'ipfa_json_test.json')

In [ ]:
print(ipfa.objective.expression)

In [ ]:
exchange_rxns = [exchange.id for exchange in ipfa.exchanges] # find exchange reactions
len(exchange_rxns)

In [ ]:
ipfa.reactions.EXC_BOTH_C00062_e

In [ ]:
ipfa.compartments

In [ ]:
# find all substrates of the reaction
biomets = ipfa.reactions.Biomass_rxn_c.metabolites #r_4041 = biomassreaction 8

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
biomass.sort_values(by='metabolite', ascending='False')

In [ ]:
import re

def kegg_to_bigg(kegg_id):
    # Make the request to the BiGG API
    response = requests.post('http://bigg.ucsd.edu/advanced_search_external_id_results',
                            data={'database_source': 'kegg.compound', 'query': kegg_id})
    # Check if the request was successful
    if response.status_code == 200:
        try:
            print(re.search(r'/models/universal/metabolites/([^"]+)', response.text).group(1))
        except AttributeError:
            print('no result')
            print(kegg_id)
    else:
        print('Error:', response.status_code)
for metabolite in biomass.ID:
    id = metabolite.id[:-2]
    kegg_to_bigg(id)

In [ ]:

'''
cobra.flux_analysis.pfba(model: cobra.Model, fraction_of_optimum: float = 1.0, objective: Union[Dict, optlang.interface.Objective, None] = None, reactions: Optional[List[cobra.Reaction]] = None) → cobra.Solution
minimizes overall fluxes
'''
opt = ipfa.optimize()
lethal = 0
frac = opt.objective_value * 0.9


In [ ]:
ipfa.metabolites.C00641_c

In [ ]:
ipfa.reactions.R02687_c

In [ ]:
'''
ipfa extra lipids in biomass

Neutral: cardiolipin, 1,2 diacyl-gycerol,
Fatty acids: Octadecanoic acid, Hexadecanoic acid, (9Z)-Octadecenoic acid, Tetradecanoic acid
'''
lipids = ['Cholesterol','Phosphatidylcholine','Phosphatidylserine','Phosphatidylethanolamine','Sphingomyelin','Triacylglycerol']
all_lps = ['cardiolipin','1,2 diacyl-gycerol','Octadecanoic acid','Hexadecanoic acid', '(9Z)-Octadecenoic acid', 'Tetradecanoic acid']
lipid_df = biomass.loc[biomass['metabolite'].isin(lipids)]
lipid_df

In [ ]:
# first  value % of total lipids containing a lipid(PLs, SM, Cholesterol, Neutral)
# 45.7*58.4 is 45.7% phospholipids of total lipid amount times 58.4% of PC in PLs
# divided by 100 to have %
# values % of specific lipid during this stage of total lipid amount
newlipid_df = lipid_df.assign(  Ring=[45.7*58.4/100,45.7*33.2/100,45.7*7.7/100,14.0,32.4,7.9*20.4/100])
newlipid_df = newlipid_df.assign(  Trophozoite=[60.1*59.8/100,60.1*28.7/100,60.1*9.1/100,8.0,22.0,10.0*17.5/100])
newlipid_df = newlipid_df.assign(  Schizont=[69.1*66.4/100,69.1*24.8/100,69.1*5.0/100,12.5,4.1,33.0*14.2/100])

from matplotlib.pyplot import cm

phases = ['Ring', 'Trophozoite', 'Schizont']

fig, ax = plt.subplots(figsize=(15,10))
color=iter(cm.rainbow(np.linspace(0,1,len(phases))))

for phase in phases:
    newlipid_df[phase] = newlipid_df[phase]/newlipid_df[phase].sum()*newlipid_df.stoichiometry.sum()
    c=next(color)
    if phase == 'Trophozoite':
        for i, txt in enumerate(newlipid_df['metabolite']):
            ax.annotate(txt, xy=(newlipid_df['stoichiometry'].iat[i], newlipid_df[phase].iat[i]))
    ax = newlipid_df.plot.scatter(x='stoichiometry', y = phase, color=c, label=phase, s=50, ax=ax)
plt.ylabel('Phases value')

In [ ]:
newlipid_df

In [ ]:
#get id from weird reaction datatype(frozenset)
def id_rct_conv(x):
    st = str(x)
    lst = st.split(',')
    react = []
    for e in lst:
        start = e.find('Reaction ')
        end = e.find(' at')
        e = e[start+len('Reaction '):end]
        react.append(e)
    return(react)

# get names of metabolites involved in reaction
def get_metas_rxn(rxn):
    biomets = ipfa.reactions.get_by_id(rxn).metabolites

    m = list(biomets.keys()) # ID of the metabolite in the reaction
    metname = [mi.name for mi in m] # names of the metabolite in the reaction
    return metname

# get id of metabolites involved in reaction
def get_id_metas_rxn(rxn):
    biomets = ipfa.reactions.get_by_id(rxn).metabolites

    m = list(biomets.keys()) # ID of the metabolite in the reaction
    return m


#get actual names of reactions(names) for list of reaction ids(x)
def nme_rct_lst(x):
    names = []
    for val in x:
        name=str(val)
        names.append(ipfa.reactions.get_by_id(name).name)
        
    return names   

#get actual fluxes of reactions(names) for list of reaction ids(x)
def flux_rct_lst(x):
    fluxes = []
    for val in x:
        fl=str(val)
        fluxes.append(ipfa.reactions.get_by_id(fl).flux)
        
    return fluxes   

#get all reactions(ID,name) in which metabolite(x) is involved converted to panda table
def print_reactions(x):
    b = ipfa.metabolites.get_by_id(x).reactions

    idnme = id_rct_conv(b)
    nm = nme_rct_lst(idnme)
    
    # table returning (this is very messy)
    import pandas as pd
    df = pd.DataFrame.from_records(map(list, zip(*[idnme,nm])),columns=['ID', 'name'])
    return df

#get all reactions(ID,name) with its corresponding flux in which metabolite(x) is involved converted to panda table
def print_rct_wth_flux(x):
    b = ipfa.metabolites.get_by_id(x).reactions
    idnme = id_rct_conv(b)
    nm = nme_rct_lst(idnme)
    fls= flux_rct_lst(idnme)
    
    # table returning (this is very messy)
    df = pd.DataFrame.from_records(map(list, zip(*[idnme,nm,fls])),columns=['ID', 'name','flux'])
    #create table with reactions that have flux
    df = df[df['flux'] > 0]
    dna_s = df.dropna( axis=1, how='any')
    
    return dna_s

#get all metabolites involed in ripfa.metabolites.get_by_id('C00269_c').summary()eaction(x) with their stoichometric factor in panda data frame
def print_stoich_reaction(x):
    # find all substrates of the reaction
    biomets = ipfa.reactions.get_by_id(x).metabolites

    s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
    m = list(biomets.keys()) # ID of the metabolite in the reaction

    # a table of metabolites and their stoichiometry

    metname = [mi.name for mi in m] # names of the metabolite in the reaction

    # panda table creating (this is very messy)
    import pandas as pd

    df = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
    return df

def summary_wth_names(print_on=True):
    # find all excretion reactions
    exrxns = ipfa.exchanges162809
    rxnids = [e.id for e in exrxns]

    # get values for their bounds
    # exrxns[1]oo
    # exrxns[1].bounds
    b = [e.flux for e in exrxns]  # flux of the reaction
    metname = [get_metas_rxn(r) for r in rxnids]  # names of the reaction
    metaids = [get_id_metas_rxn(r) for r in rxnids]
    # table printing (this is very messy)
    df=pd.DataFrame.from_records(map(list, zip(*[metaids,metname,b])),columns=['IDs','Names','flux'])

    #out flux sorted
    outflux=df[df['flux'] > 0.1E-04]
    outflux=outflux.sort_values('flux',ascending=False)
    outflux= outflux.reset_index(drop=True)
    
    #in flux sorted
    influx=df[df['flux'] < -0.1E-04]
    influx=influx.sort_values('flux',ascending=True)
    influx = influx.reset_index(drop=True)
    #joining of dataframes
    dr=influx.join(outflux,lsuffix='_IN Flux',rsuffix='_OUT Flux')
    #make NaN to empty strings
    show = dr.replace(np.nan, '', regex=True)
    #make two headings out of (Names_IN Flux) Names as sub and IN Flux above
    show.columns = pd.MultiIndex.from_tuples([tuple(list(reversed(c.split('_')))) for c in show.columns])
    if print_on:
        print(show)
    return show

In [ ]:
print_rct_wth_flux('C00157_r')

In [ ]:
print_reactions('C00157_r')

In [ ]:
# read file to get all reactions in subsystem of interest
table = pd.read_excel('ipfa_raven_input.xlsx', header=0)
# these are the 'subsytems' we want to concentrate on, list not complete
subsystems = ['Fatty acid degradation', 'Glycerophospholipid metabolism',
              'Glycerolipid metabolism', 'Ether lipid metabolism', 'N-Glycan biosynthesis',
              'Lipoic acid metabolism', 'Phosphonate and phosphinate metabolism',
              'Sphingolipid metabolism', 'Fatty acid metabolism / biotin (vit B7) dependent',
              'Fatty acid metabolism', 'Isoprenoid metabolism', 'Steroid metabolism',
              'Glycosylphosphatidylinositol(GPI)-anchor biosynthesis']

table = table.loc[:, 'ID':] # cut empty column
table = table[table['SUBSYSTEM'].isin(subsystems)] # select only rows with right subsystem

In [ ]:
table.sort_values(by='ID')

In [ ]:
print('complete model: ', ipfa.optimize())
summary_wth_names()
#look at Oxyhemoglobin path in ipfa

In [ ]:
newlipid_df

In [ ]:
# optimize iPFA for different phases and their corrosponding lipid values 
# stoichiometry is iPFA standard values
ex_flux_dict = {}
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']
for phase in phases:
    with ipfa:
        # loop to create and implement new stoichiometric values
        for meta in newlipid_df.ID:
            # creat new float value from table
            zero = float(newlipid_df.loc[newlipid_df['ID'] == meta]['stoichiometry'])
            new_stoich = float(newlipid_df.loc[newlipid_df['ID'] == meta][phase])
            # implement new value in biomass function
            ipfa.reactions.Biomass_rxn_c.add_metabolites({meta:new_stoich-zero})
        
        # print solution value and boundary fluxes
        print(phase+' model: ', ipfa.optimize().objective_value)

        print(ipfa.reactions.Biomass_rxn_c)
        # add to lipid-reaction-table the calculated fluxes through them 
        table[phase + '_Flux'] = flux_rct_lst(table['ID'].tolist())    
        
        
        #for i in newlipid_df.ID:
        #    exe='print(ipfa.metabolites.'+ str(i) +'.summary(threshold=0.00002))'
        #    exec(exe)
        ex_flux_dict[phase] = summary_wth_names(print_on=False)

In [ ]:
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']

stoich = ex_flux_dict['stoichiometry']['OUT Flux']['flux'][ex_flux_dict['stoichiometry']['OUT Flux']['flux'] != '']

out_dataframe = pd.DataFrame()
in_dataframe = pd.DataFrame()

for current in ['IN','OUT']:
    for phase in phases:
        mini_frame = pd.DataFrame()
        mini_frame[phase +'_Names'] = ex_flux_dict[phase][current+' Flux']['Names'][ex_flux_dict[phase][current+' Flux']['Names'] != '']

        for i, name in enumerate(mini_frame[phase +'_Names']):
            mini_frame[phase +'_Names'].iloc[i] = name[0]

        phase_dataframe = ex_flux_dict[phase][current+' Flux']['flux'][ex_flux_dict[phase][current+' Flux']['flux'] != '']
        mini_frame[phase] = phase_dataframe

        mini_frame = mini_frame.sort_values(by=phase +'_Names', ascending='False',ignore_index=True)
        if current == 'OUT':
            out_dataframe = mini_frame.join(out_dataframe, how='outer')
        else:
            in_dataframe = mini_frame.join(in_dataframe, how='outer')

melted_df = out_dataframe.melt(id_vars=['Schizont_Names', 'Trophozoite_Names', 'Ring_Names', 'stoichiometry_Names'],
                    value_vars=['Schizont', 'Trophozoite', 'Ring', 'stoichiometry'],
                    var_name='Stage',
                    value_name='Value')
name_list = []
for i,stage in enumerate(melted_df['Stage']):
    name_list.append(melted_df[stage+'_Names'].iloc[i])
melted_df['name']=name_list
out_df = melted_df.drop(['Schizont_Names', 'Trophozoite_Names', 'Ring_Names', 'stoichiometry_Names'],axis=1)
out_df = out_df.dropna()

melted_df = in_dataframe.melt(id_vars=['Schizont_Names', 'Trophozoite_Names', 'Ring_Names', 'stoichiometry_Names'],
                    value_vars=['Schizont', 'Trophozoite', 'Ring', 'stoichiometry'],
                    var_name='Stage',
                    value_name='Value')
name_list = []
for i,stage in enumerate(melted_df['Stage']):
    name_list.append(melted_df[stage+'_Names'].iloc[i])
melted_df['name']=name_list
in_df = melted_df.drop(['Schizont_Names', 'Trophozoite_Names', 'Ring_Names', 'stoichiometry_Names'],axis=1)
in_df = in_df.dropna()

In [ ]:
def plot_dataframe(df, description='',value_str=''):
    grouped_df = df.groupby(['name','Stage']).mean().reset_index()
    # Create a colormap and markers for the stages
    colormap = plt.cm.get_cmap('tab10')
    markers =  ("s","D","^","o")
    marks = iter(markers)

    # Plot the scatter plot
    fig, ax = plt.subplots(figsize=(7,15))

    # Iterate over each stage
    for i, stage in enumerate(grouped_df['Stage'].unique()):
        stage_data = grouped_df[grouped_df['Stage'] == stage]
        ax.scatter(stage_data[value_str+'Value'],stage_data['name'], color=colormap(i), label=stage, marker=next(marks))

    # Add labels and title
    ax.set_ylabel('Metabolite')
    ax.set_xlabel(description+'ward-Flux '+ value_str +' Value')
    #ax.set_title('Values by Name with Stage')

    # Add a legend
    ax.legend(grouped_df['Stage'].unique(), bbox_to_anchor=(0., 1.001, 1., .102), loc=3,ncol=4, mode="expand",fancybox = True, borderaxespad=0.)
    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=45)

    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
out_df

In [ ]:
plot_dataframe(out_df,'Out')

In [ ]:
## Group the DataFrame by name
grouped_df = out_df.groupby(['name'])

# Define a function to calculate the relative value
def calculate_relative_value(x):
    if 'stoichiometry' in x['Stage'].values:
        stoichiometry_value = x.loc[x['Stage'] == 'stoichiometry', 'Value'].values[0]
        x['Relative_Value'] = x['Value'] / stoichiometry_value
    else:
        x['Relative_Value'] = 0
    return x

# Apply the transformation to each group
transformed_df = grouped_df.apply(calculate_relative_value)
transformed_df.drop('Value',axis=1)

plot_dataframe(transformed_df,'Out','Relative_')

In [ ]:
plot_dataframe(in_df,'IN-')

In [ ]:
## Group the DataFrame by name
grouped_df = in_df.groupby(['name'])

# Define a function to calculate the relative value
def calculate_relative_value(x):
    if 'stoichiometry' in x['Stage'].values:
        stoichiometry_value = x.loc[x['Stage'] == 'stoichiometry', 'Value'].values[0]
        x['Relative_Value'] = x['Value'] / stoichiometry_value
    else:
        x['Relative_Value'] = 0
    return x

# Apply the transformation to each group
transformed_df = grouped_df.apply(calculate_relative_value)
transformed_df.drop('Value',axis=1)

plot_dataframe(transformed_df,'IN-','Relative_')

In [ ]:

color=iter(cm.hsv(np.linspace(0,1,len(phases)+1)))


In [ ]:
phasess=['stoichiometry_Flux','Ring_Flux','Trophozoite_Flux','Schizont_Flux']
checker = table[phasess].sum(axis=1).abs()
with_flux_table = table[checker >= 0.1E-010]
with_flux_table.drop(['GENE ASSOCIATION','REPLACEMENT ID','COMPARTMENT','MIRIAM','OBJECTIVE'],axis=1)

In [ ]:
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']
markers =  ("s","D","^","o")
marks = iter(markers)
colormap = plt.cm.get_cmap('tab10')
annotate = False

fig, ax = plt.subplots(figsize=(10,30))
for i,phase in enumerate(phases):
    if annotate:
        for i, txt in enumerate(with_flux_table['ID']):
            ax.annotate(txt, xy=(with_flux_table['stoichiometry_Flux'].iat[i], table[phase+'_Flux'].iat[i]))
    ax = with_flux_table.plot.scatter (x=phase+'_Flux', y ='ID' , color=colormap(i), label=phase, marker=next(marks), s=50, ax=ax)
    #ax.set_xlim(min(table['stoichiometry_Flux'])-0.01, max(table['stoichiometry_Flux'])+0.01)

plt.xlabel('Phases flux-values')
plt.legend(phases,fontsize=17,bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=4, mode="expand",fancybox = True, borderaxespad=0.)
plt.tight_layout()

In [ ]:
fluxi=[]
for phase in phases:
    fluxi.append(phase+'_Flux')
    
#checker = table[fluxi].sum(axis=1)
#newtable = table[checker!=0]

fig, ax = plt.subplots(figsize=(10,40))
with_flux_table.plot.barh(x='ID', y =fluxi, label=phases, ax=ax)
ax.set_xlim(min(with_flux_table['stoichiometry_Flux'])-0.01, max(with_flux_table['stoichiometry_Flux'])+0.01)

plt.xlabel('Phases flux-values')
plt.ylabel('Reaction IDs')
plt.legend(phases, fontsize = 17, bbox_to_anchor = (0., 1.001, 1., .102), loc = 3, ncol = 4, mode = "expand", fancybox = True, borderaxespad = 0.)
plt.tight_layout()

In [ ]:
with_flux_table

In [ ]:
stacked_data = with_flux_table[fluxi].apply(lambda x: x*100/max(abs(x)), axis=1)
stacked_data['ID'] = with_flux_table['ID']
stacked_data['NAME'] = with_flux_table['NAME']
fig, ax = plt.subplots(figsize=(17,35))
stacked_data.plot.barh(x = 'NAME', y = fluxi , label = phases, ax = ax)

plt.xlabel('Flux-value in [%] to max_flux_value in Phases')
plt.ylabel('Reaction ID')
plt.legend(phases, fontsize = 16, bbox_to_anchor = (0., 1.001, 1., .102), loc = 3, ncol = 4, mode = "expand", fancybox = True, borderaxespad = 0.)
plt.tight_layout()

In [ ]:
ipfa.reactions.R07770_a

In [ ]:
ipfa.reactions.R07771_a

In [ ]:
ipfa.reactions.R01280_a

In [ ]:
ipfa.reactions.R09380_a

In [ ]:
ipfa.reactions.R00842_a

## Analysis of lipid biomass network

In [ ]:
ipfa.optimize()

In [ ]:
print(ipfa.metabolites.C00157_r.name)
ipfa.metabolites.C00157_r.summary()

In [ ]:
ipfa.reactions.R01321_r

In [ ]:
print(ipfa.metabolites.C00249_c.name)
ipfa.metabolites.C00249_c.summary()

In [ ]:
ipfa.reactions.R01280_c

In [ ]:
lipids_biomass = ['C00416_c','C00641_c','C00249_c', 'C00157_r', 'C00350_r','C02737_r','C00550_c','C00187_c','C00422_c','C00712_c','C01530_c','C06424_c']
for lips in lipids_biomass:
    print(ipfa.metabolites.get_by_id(lips).name)
    print(ipfa.metabolites.get_by_id(lips).summary())
    print(print_reactions(lips))

In [ ]:
reactions = ['R01321_r','R02057_R02038_r','R07377_r','R01891_c']
for reaction in reactions:
    print(ipfa.reactions.get_by_id(reaction))

In [ ]:
ipfa.metabolites.get_by_id('C00641_r').summary()

In [ ]:
ipfa.reactions.get_by_id('R02057_R02038_r')

In [ ]:
ipfa.reactions.get_by_id('R01799_c')

In [ ]:
print(ipfa.metabolites.get_by_id('C00269_c').name)
ipfa.metabolites.get_by_id('C00269_c').summary()

In [ ]:
ipfa.reactions.get_by_id('R02030_c')

In [ ]:
print(ipfa.metabolites.get_by_id('C05980_c').name)
ipfa.metabolites.get_by_id('C05980_c').summary()

In [ ]:
ipfa.reactions.get_by_id('R07390_c')

In [ ]:
ipfa.metabolites.get_by_id('C00344_c').summary()

In [ ]:
ipfa.reactions.get_by_id('R01802_c')

In [ ]:
ipfa.reactions.get_by_id('R02240_c')

In [ ]:
ipfa.reactions.get_by_id('R03435_c')

In [ ]:
ipfa.reactions.get_by_id('R01280_c')

In [ ]:
ipfa.reactions.get_by_id('R02057_R02038_r')

In [ ]:
ipfa.reactions.get_by_id('R01891_c')

In [ ]:
ipfa.reactions.get_by_id('R01321_r')

In [ ]:
ipfa.reactions.get_by_id('R07377_r')

In [ ]:
print(ipfa.metabolites.C00002_c.name)
ipfa.metabolites.C00002_c.summary()

In [ ]:
print(ipfa.metabolites.C00154_c.name)
ipfa.metabolites.C00154_c.summary()

In [ ]:
ipfa.reactions.R01281_c

In [ ]:
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(30,30))

for i,phase in enumerate(phases):
    
    my_dataframe = ex_flux_dict[phase]['OUT Flux']['flux'][ex_flux_dict[phase]['OUT Flux']['flux'] != '']
    my_dataframe.plot.pie(y='flux',labels = ex_flux_dict[phase]['OUT Flux']['Names'], ax= axes.flat[i])
    
    axes.flat[i].set_xlabel(phase+'-phase flux-values')
    axes.flat[i].set_ylabel('')

plt.tight_layout()

In [ ]:
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(30,30))

for i,phase in enumerate(phases):
    
    helper = ex_flux_dict[phase]['IN Flux']['flux']*(-1)
    helper.plot.pie(y='flux', labels = ex_flux_dict[phase]['IN Flux']['Names'], ax= axes.flat[i])
    
    axes.flat[i].set_xlabel(phase+'-phase flux-values')
    axes.flat[i].set_ylabel('')

plt.tight_layout()

In [ ]:
phases = ['stoichiometry', 'Ring', 'Trophozoite', 'Schizont']
color=iter(cm.rainbow(np.linspace(0,1,len(phases))))

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(30,30))

for i,phase in enumerate(phases):
    c=next(color)
    ex_flux_dict[phase]['IN Flux'].plot.barh(x='Names', y ='flux',label= phase, color=c, ax= axes.flat[i])
    #ax.set_xlim(min(table['stoichiometry_Flux'])-0.01, max(table['stoichiometry_Flux'])+0.01)
    axes.flat[i].set_ylabel(phase+'-phase flux-values')
    #axes.flat[i].legend(phase,fontsize=17,bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=1, mode="expand",fancybox = True, borderaxespad=0.)

plt.tight_layout()



In [ ]:
reactionstable = pd.read_csv('~/first-lipid-model/odemodel/reactions.csv', header=0)
reactionstable

In [ ]:
ipfarid = [reactionid[:] for reactionid in table['ID']]#
modelsrid = list(reactionstable['ID'])
#check if model ids are in iPFA reaction ids since they are shortend
matching = [reaction for reaction in ipfarid if any(modelid in reaction for modelid in modelsrid)]
len(matching)

In [ ]:
newtable = table[table['ID'].isin(matching)]
newtable = newtable.sort_values(by='ID', ascending='False',ignore_index=True)
newtable.drop(['OBJECTIVE', 'COMPARTMENT', 'MIRIAM'], axis=1)

In [ ]:
newtable['NAME']

In [ ]:
matching2 = [reaction for reaction in modelsrid if any(reaction in modelid for modelid in ipfarid)]
phasess=['stoichiometry_Flux','Ring_Flux','Trophozoite_Flux','Schizont_Flux']
rct_values_dict = {}
for r in matching2:
    allrs = [reaction for reaction in ipfarid if r in reaction]
    
    shorttable = newtable[newtable['ID'].isin(allrs)]
    values = shorttable[phasess].sum()
    
    rct_values_dict[r]=values


In [ ]:
#read dot file of lasse and rewrite to add the flux values of ipfa
wholetxt=""
with open("/home/maxim/PhD/Lipid-Model/Model/visualization/model.dot", "r") as f:
    for cnt, line in enumerate(f):
        #get reaction node descriptor line
        if line[:6] in rct_values_dict.keys() and line[7] == "[":
            text = ""
            lins = line
            #rewrite line with values of fba with maier data
            for i, value in enumerate(rct_values_dict[line[:6]]):
                text += ("\n {} = {:.4f}".format(rct_values_dict[line[:6]].index[i],value))
                
            new = line[:-4] + text + '"];\n'
            wholetxt += new
        #else write line as it is
        else:
            wholetxt += line
# write dot file            
with open('file.dot', 'w') as file:
    file.write(wholetxt)

In [ ]:
fluxi=[]
for phase in phases:
    fluxi.append(phase+'_Flux')
# get all shared reactions even if there is no flux in iPFA


fig, ax = plt.subplots(figsize=(12,10))
newtable.plot.barh(x='NAME', y =fluxi , label=phases, ax=ax)
#ax.set_xlim(min(with_flux_table['stoichiometry_Flux'])-0.01, max(with_flux_table['stoichiometry_Flux'])+0.01)
#Same means reaction happens in two different compartments 
plt.xlabel('Phases flux-values')
plt.ylabel('Reaction IDs')
plt.legend(phases, bbox_to_anchor = (0., 1.001, 1., .102), loc = 3, ncol = 4, mode = "expand", fancybox = True, borderaxespad = 0.)
plt.tight_layout()

In [ ]:
stacked_data = newtable[fluxi].apply(lambda x: x*100/max(abs(x)), axis=1)
stacked_data['NAME'] = newtable['NAME']

fig, ax = plt.subplots(figsize=(12,10))
stacked_data.plot.barh(x ='NAME', y = fluxi , label = phases, ax = ax)

plt.xlabel('Flux-value in correlation to max_flux_value(Phases)')
plt.ylabel('Reaction ID')
plt.legend(phases, bbox_to_anchor = (0., 1.001, 1., .102), loc = 3, ncol = 4, mode = "expand", fancybox = True, borderaxespad = 0.)
plt.tight_layout()

In [ ]:
import requests
def kegg_by_name(metabolite_name):

    # Query the KEGG API for the metabolite's information
    response = requests.get(f"http://rest.kegg.jp/find/compound/{metabolite_name}")
    if response.ok:
        # If the query was successful, retrieve the KEGG ID from the response
        ids=[]
        for line in response.text.strip().split("\n"):
            metaid, aliases = line.split('\t')
            metaid = metaid.split(':')[1]
            aliases = [alias.strip() for alias in aliases.split(';')]
            match = [alias for alias in aliases if alias == metabolite_name]
            if match!=[]:
                ids.append(metaid)
        print(f"The KEGG ID for {metabolite_name} is {ids}")
        return ids[0]
    else:
        # If the query was not successful, print an error message
        print(f"Error querying KEGG API: {response.status_code} {response.reason}")

def convert_bigg_to_kegg(bigg_id, entity):
    '''Function to translate BiGG reaction ID into KEGG ID'''
    # Make a GET request to the BiGG API
    response = requests.get(f'http://bigg.ucsd.edu/api/v2/universal/{entity}/{bigg_id}')
    linker = 'Reaction'
    if entity == 'metabolites':
        linker = 'Compound'
    # Extract the KEGG ID from the response JSON
    try:
        kegg_id = response.json()['database_links'][f'KEGG {linker}'][0]['id']
        print(response.json()['name'])
        return kegg_id
    except (KeyError, IndexError):
        print(f'No KEGG ID found for BiGG ID: {bigg_id}')
        if entity == 'metabolites':
            return kegg_by_name(response.json()['name'])
        else:
            print(response.json()['name'])
        return None

In [ ]:
ribo_rcts = ['RBFK','RIBFLVt2','ACP1']
kegg_ribo = [convert_bigg_to_kegg(x, 'reactions') for x in ribo_rcts]

In [ ]:
anti_metas = ['ribflv','ncam','pydxn','thm']
#['C00378']
tester = [convert_bigg_to_kegg(x, 'metabolites') for x in anti_metas]

In [ ]:
tester

In [ ]:
ipfa = cobra.io.read_sbml_model("ipfasbml.xml")

In [ ]:
def run_fba_without_rxns_metabolite(model, meta):
    
    rxn_set = model.metabolites.get_by_id(meta).reactions
    rxns = [rxn.id for rxn in rxn_set]
    f = None
    with model:
        model.remove_reactions(rxns)
        f = model.optimize().objective_value
    return f 
frac = 0.9 * ipfa.optimize().objective_value

anti_metas = ['ribflv','ncam','pydxn','thm']
tester = [convert_bigg_to_kegg(x, 'metabolites') for x in anti_metas]
# 'ribflv[c]' translates to C00255_c
for meta in tester:
    string=meta +'_c'
    #rxns = ipfa.metabolites.get_by_id(string).reactions
    value = run_fba_without_rxns_metabolite(ipfa, string)

    if value > frac:
        print(f'Model grows with antimetabolite for {string}')
    else:
        print(f'Passes, test for {string}')
        

In [ ]:
exchange_rxns = [exchange.id for exchange in ipfa.exchanges] # find exchange reactions
with ipfa:
    for reaction in exchange_rxns:
        ipfa.reactions.get_by_id(reaction).bounds=(-1000,0) # block all exchange reactions
    meta = convert_bigg_to_kegg('atp', 'metabolites')
    ipfa.add_boundary(ipfa.metabolites.get_by_id(meta+'_c'), type='demand') # add ATP demand reaction
    ipfa.objective_coefficient = 1 # set objective coefficient to 1
    value = ipfa.optimize().objective_value # optimize the minimal medium for the modified model
    if value >= frac:
        print('Model produces ATP when all import reactions are blocked')
    else:
        print(f'Passes, no ATP production, solution: {value}, when all import reactions are blocked')

In [ ]:
purines = ['HXAN','GUA','DIMP','INS','ADN','DAD_2','DGSN','DIN','GSN','PAP','XAN']
kegg_purines = [convert_bigg_to_kegg(x.lower(), 'metabolites') for x in purines]
kegg_purines

In [ ]:
kegg_purines[-2]='C00170'
kegg_purines

In [ ]:
for meta in kegg_purines:
    rxn_set = ipfa.metabolites.get_by_id(meta+'_c').reactions
    exchange_rxns = [exchange.id for exchange in rxn_set]
    print(exchange_rxns)

In [ ]:
table = pd.read_excel('ipfa_raven_input.xlsx', header=0)

In [ ]:
eq_rxns = table[table['SUBSYSTEM'].isin(['Purine metabolism'])]['EQUATION'].to_list()

In [ ]:
def split_reaction_string(reaction_str):
    metabolites = reaction_str.split("<=>")
    reactants = metabolites[0].strip().split("+")
    products = metabolites[1].strip().split("+")
    return reactants + products

def flatten(temp_list):
    for ele in temp_list:
        if type(ele) == list:
            flatten(ele)
        else:
            new_list.append(ele.strip(' '))

all_metas = [split_reaction_string(x) for x in eq_rxns]
new_list=[]
flatten(all_metas)
purines_set = set(new_list).copy()

In [ ]:
eq_ex_rxns = table[table['SUBSYSTEM'].isin(['Transport between c and e'])]['EQUATION'].to_list()
ex_metas = [split_reaction_string(x) for x in eq_ex_rxns]
new_list=[]
flatten(ex_metas)
ex_all = set(new_list).copy()
list(ex_all.intersection(purines_set))

In [ ]:
purines_ex = ['Purine[c]', 'Deoxyadenosine[c]', 'Adenosine[c]', 'L-Aspartate[c]',
 'Purine deoxyribonucleoside[c]', 'Xanthosine[c]', 'L-Homocysteine[c]', 'Xanthine[c]',
 'Selenohomocysteine[c]',
 'Guanosine[c]',
 'Thioredoxin disulfide[c]',
 'Se-Adenosyl-L-selenohomocysteine[c]',
 'Thioredoxin[c]',
 'Se-Adenosylselenomethionine[c]',
 'Hypoxanthine[c]',
 'S-Adenosyl-L-homocysteine[c]',
 'Fumarate[c]',
 'Inosine[c]',
 'L-Selenomethionine[c]',
 '(-)-Ureidoglycolate[c]',
 'Deoxyguanosine[c]',
 'Allantoate[c]',
 'Guanine[c]',
 'S-Adenosylmethioninamine[c]',
 'N-D-Ribosylpurine[c]',
 'L-Glutamate[c]',
 'Adenine[c]',
 'L-Glutamine[c]',
 'Deoxyinosine[c]',
 'S-Adenosyl-L-methionine[c]',
 "5'-Methylthioadenosine[c]"]

In [ ]:
purine_ipfa = [kegg_by_name(x[:-3]) for x in purines_ex]
purine_ipfa[-7] = 'C15586'

In [ ]:
purine_ipfa=['C15587','C00559','C00212',
             'C00049','C20463','C01762',
             'C00155','C00385','C05698',
             'C00387','C00343','C05692',
             'C00342','C05691','C00262',
             'C00021','C00122','C00294',
             'C05335','C00603','C00330',
             'C00499','C00242','C01137',
             'C15586','C00025','C00147',
             'C00064','C05512','C00019',
             'C00170']


In [ ]:
purine_ex_rxns=[]
for meta in purine_ipfa:
    try:
        #print(meta)
        rxn_set = ipfa.metabolites.get_by_id(meta+'_c').reactions
        exchange_rxns = [exchange.id for exchange in rxn_set]
        purine_ex_rxns.append(f'T_c_to_e_{meta}')
        #print(exchange_rxns)
    except(KeyError):
        print(f'No cytosolic {meta}')
purine_ex_rxns

In [ ]:
def boundary_chnge_run_fba(model, rxns, bounds):
    f = None
    with model:
        for i,reaction in enumerate(rxns):
            model.reactions.get_by_id(reaction).bounds=bounds[i] # change boundaries
        #print(ipfa.medium)
        f = model.optimize().objective_value
        #print(summary_wth_names())
    return f
lethal = 0.0
for i in range(len(purine_ex_rxns)):
    tmp = purine_ex_rxns.pop(i)
    bound = [(-0,0)]*len(purine_ex_rxns)
    value = boundary_chnge_run_fba(ipfa, purine_ex_rxns, bound)
    print(value)
    purine_ex_rxns.insert(i, tmp)
    bound = [(0,0)]*len(purine_ex_rxns)
    value = boundary_chnge_run_fba(ipfa, purine_ex_rxns, bound)
    if value > lethal:
        print(f'Purine {purines_ex[i]} not necessary for growth')
    else:
        print(f'Purine {purines_ex[i]}')

In [ ]:
# Leaky metabolites
exchange_rxns = [exchange.id for exchange in ipfa.exchanges] # find exchange reactions
leaky = np.zeros(len(exchange_rxns))
for i,rxn in enumerate(exchange_rxns):
    with ipfa:
        for reaction in exchange_rxns:
            ipfa.reactions.get_by_id(reaction).bounds=(0,1000) # block all exchange reactions
        ipfa.objective = {ipfa.reactions.get_by_id(rxn): 1}
        leaky[i] = ipfa.optimize().objective_value
# Print leaky reactions
print(np.asarray(exchange_rxns)[np.where(leaky>1e-9)])
print(leaky[np.where(leaky>1e-9)])

In [ ]:
print(ipfa.metabolites.get_by_id('C00080_e').name)
ipfa.reactions.EXC_BOTH_C00080_e

In [ ]:
def boundary_chnge_run_fba(model, rxns, bounds):
    f = None
    with model:
        for i, reaction in enumerate(rxns):
            model.reactions.get_by_id(reaction).bounds=bounds[i] # change boundaries
        #print(ipfa.medium)
        f = model.optimize().objective_value
        #print(summary_wth_names())
    return f
# No 'D-Glucose' import -> D-Glucosamine C00329 compensates,
# if both out -> fructose compensates, probably wrong representation of sugar metabolism for asexual 
rxns_glu = ['T_c_to_e_C00031','T_c_to_e_C00329']
bound = [(0,0)]*len(rxns_glu)
value = boundary_chnge_run_fba(ipfa, rxns_glu, bound)
print(value)

In [ ]:
# Important AA
# L-Isoleucine, L-Tyrosine, L-Methionine, L-Proline	L-Glutamine L-Glutamate 5-semialdehyde
aa_metas = ['C00407', 'C00082','C00073', 'C00148','C00064','C01165','C00568']
aa_metas_ex_rxns=[]
for aa_meta in aa_metas:
    try:
        #print(meta)
        rxn_set = ipfa.metabolites.get_by_id(aa_meta+'_c').reactions
        exchange_rxns = [exchange.id for exchange in rxn_set]
        aa_metas_ex_rxns.append(f'T_c_to_e_{aa_meta}')
        print(exchange_rxns)
    except(KeyError):
        print(f'No cytosolic {aa_meta}')

for i,rxn_t in enumerate(aa_metas_ex_rxns):
    #print(rxn_t)
    bound = [(-0.0,0)]*len([rxn_t])
    value = boundary_chnge_run_fba(ipfa, [rxn_t], bound)
    name = ipfa.metabolites.get_by_id(aa_metas[i]+'_c').name
    if value <= frac:
        print(f'Growth reduced without {name}, solution at {value}')
    else:
        print(f'No growth reduction with no {name}')

In [ ]:
prods = ipfa.reactions.HBDG_c.products
[pord.name for pord in prods]

In [ ]:
ipfa.optimize()
ipfa.summary(fva=0.95)

In [ ]:
ipfa.metabolites.C00078_c.summary(fva=0.99)

In [ ]:
pfba_solution = cobra.flux_analysis.pfba(ipfa)

In [ ]:
pfba_solution

In [ ]:
blo = cobra.flux_analysis.find_blocked_reactions(ipfa)
blo # more than metnetx found

In [ ]:
def minimal_medium(model, indispensible=False, cols=11):
    # Find maximal growth rate
    max_growth = model.slim_optimize()
    # Find minimal media (11) that have defined growth rate
    # Exchange reactions are found, each column one possible medium 
    medium_df = cobra.medium.minimal_medium(model, max_growth, minimize_components=cols)
    # Get metabolites of the exchange reactions
    mets = []
    for ind in medium_df.index:
        met = model.reactions.get_by_id(ind).metabolites    
        mets = mets + list(met.keys())
    # Get name of the metabolites
    name_mets = [x.name for x in mets]
    # Set in dataframe
    medium_df['name']=name_mets
    # Return if indispensible only metabolites that are present in every solution
    if indispensible:
        return medium_df[(medium_df!=0).all(axis=1)]
    # Else all solutions returned
    else:
        return medium_df


In [ ]:
medium = minimal_medium(ipfa,indispensible=0, cols=15)

In [ ]:
medium